# Exploring parameter "a" according to Yimsiri and Soga (2010)

The aim of this notebook is to calculate parameter "a" as described in Yimsiri and Soga (2010) using the Fabrictensor and the Eigenvalues. <br>
The packing structure of spherical particles can be discribed using the fabric tensor. The load transfer of the assembly is strongly dependant on contact orientations. The force tensor in the contact normal was calculated for contacts with contact force > 0: <br>
$F_{ij}=\frac{1}{N_c}\sum\limits _{N_c} ^{} n_i n_j$
<br>
Yimsiri and Soga (2010) defined parameter "a" that represents the corss-aniisotropic fabric condtion of the fabric tensor with a single parameter. 
$F_{ij} = \begin{pmatrix} \frac{3a-5}{5(a-3)} & 0 & 0\\ 0 & \frac{3a-5}{5(a-3)} & 0\\ 0 & 0 & \frac{-(5+a)}{5(a-3)} \end{pmatrix}$
- a>0, the contact normal of the particles in the assebly tend to coordinate in the vertical direction 
- a<0, the contact normal of the particles in the assebly tend to coordinate in the hoizontal direction

In [ ]:
# Import Libraries
import warnings
warnings.filterwarnings('ignore')

import re
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols, Eq, solve

from basicFunctions.get_fabric_tensor import get_fabric_tensor_cn

In [ ]:
# FIGURE
# Define Plot Fontsizes
TF = 18     # Text font
LGF = 12   # Legend Font Size
LBF = 12    # Label Font Size
TS = 12     # Tick Size
lw1 = 3

# Defining Path
path = r'F:\CyclicLoading\Cyclicmean\TX300_FC0p25to0p25_amp60\merged_data\SortedData\0_StartingPos\contact'

## Function for parameter "a"

In [ ]:
def get_a_yimsiri_soga(CNfabrictensor: np.array) -> float:
    """
    This function defines a as defined in Yimsiri and Soga (2010)
    DEM analysis of soil fabric effects on behaviour of sand
    """
    # Getting Eigenvalues from fabric tensor
    EigenValues = np.linalg.eig(CNfabrictensor)
    EigenValues = EigenValues[0] # Only want the eigenvalues and not the associated eigenvectors
    
    # Solvig using Fabric tensor 
    a11 = symbols('a11')
    a12 = symbols('a12')
    a13 = symbols('a13')
    eq11 = Eq((3*a11-5)/(5*(a11-3))-CNfabrictensor[0,0])
    eq12 = Eq((3*a12-5)/(5*(a12-3))-CNfabrictensor[1,1])
    eq13 = Eq((-(5+a13))/(5*(a13-3))-CNfabrictensor[2,2])
    
    # Equations using eigenvalues 
    a21 = symbols('a21')
    a22 = symbols('a22')
    a23 = symbols('a23')   
    eq21 = Eq((3*a21-5)/(5*(a21-3))-EigenValues[0])
    eq22 = Eq((3*a22-5)/(5*(a22-3))-EigenValues[1])
    eq23 = Eq((-(5+a23))/(5*(a23-3))-EigenValues[2])
    
    # Solving equations for a
    a_val11 = solve(eq11)[0]
    a_val12 = solve(eq12)[0]
    a_val13 = solve(eq13)[0]

    a_val21 = solve(eq21)[0]
    a_val22 = solve(eq22)[0]
    a_val23 = solve(eq23)[0]
    
    # Getting average value of a
    a1 = (a_val11 + a_val12 + a_val13)/3
    a2 = (a_val21 + a_val22 +  a_val23)/3

    return a1, a2, CNfabrictensor, EigenValues

## Processing Data 

In [ ]:
# Getting names and step number of contact dump files form selected directory 
os.chdir(path)

contact_files = os.listdir()

step_number = []
for contact_file in contact_files:
    step_number.append(re.findall('\d+', contact_file))
step_number = [int(step[0]) for step in step_number] # Convert string to integer

yimsiri_soga_a_value = pd.DataFrame(data = {'file_name':contact_files, 'step_number':step_number})
yimsiri_soga_a_value = yimsiri_soga_a_value.sort_values(by = ['step_number'], ascending = True)

In [ ]:
# Initiating lists
a_FTmean = []
a_EVmean = []
fabrictensor1 = []
fabrictensor2 = []
fabrictensor3 = []
eigenval1 = []
eigenval2 = []
eigenval3 = []

# Iterating trough dump files
for contact_file in yimsiri_soga_a_value.file_name:

    # Loading contact file
    contact_data = pd.read_csv(
            contact_file,
            skiprows = 9,
            delimiter = ' ',
            index_col = False,
            header = None
        )

    # Getting Fabric Tensor
    CNfabrictensor = get_fabric_tensor_cn(contact_data)

    # Getting mean a
    a_FT, a_EV, CNfabrictensor, EigenValues = get_a_yimsiri_soga(CNfabrictensor)
    
    # Appending to list 
    a_FTmean.append(a_FT)
    a_EVmean.append(a_EV)
    fabrictensor1.append(CNfabrictensor[0,0])
    fabrictensor2.append(CNfabrictensor[1,1])
    fabrictensor3.append(CNfabrictensor[2,2])
    eigenval1.append(EigenValues[0])
    eigenval2.append(EigenValues[1])
    eigenval3.append(EigenValues[2])

In [ ]:
# Adding values to df 
yimsiri_soga_a_value['a_FTmean'] = a_FTmean
yimsiri_soga_a_value['a_EVmean'] = a_EVmean
yimsiri_soga_a_value['eigenvalue1'] = eigenval1
yimsiri_soga_a_value['eigenvalue2'] = eigenval2
yimsiri_soga_a_value['eigenvalue3'] = eigenval3
yimsiri_soga_a_value['fabrictensor1'] = fabrictensor1
yimsiri_soga_a_value['fabrictensor2'] = fabrictensor2
yimsiri_soga_a_value['fabrictensor3'] = fabrictensor3

yimsiri_soga_a_value.insert(0, 'cycle_number', np.arange(0,len(yimsiri_soga_a_value)))

## Visualizing 

In [ ]:
yimsiri_soga_a_value.head(20)

### Parameter "a"

In [ ]:
plt.figure(figsize = (15,7))
plt.subplot(1,2,1)
plt.plot(yimsiri_soga_a_value.cycle_number,yimsiri_soga_a_value.a_FTmean, color = 'purple', lw = 2)
plt.xlabel('Cycle number (N)', fontsize = LBF)
plt.ylabel('a value from Yimsiri & Soga (2010)', fontsize = LBF)
plt.gca().tick_params(which='major', labelsize=TS)
plt.title('"a" using fabrictensor')
plt.xlim(0,)
plt.grid()

plt.subplot(1,2,2)
plt.plot(yimsiri_soga_a_value.cycle_number,yimsiri_soga_a_value.a_EVmean, color = 'darkcyan', lw = 2)
plt.xlabel('Cycle number (N)', fontsize = LBF)
plt.ylabel('a value from Yimsiri & Soga (2010)', fontsize = LBF)
plt.gca().tick_params(which='major', labelsize=TS)
plt.title('"a" using eigenvalues')
plt.xlim(0,)
plt.grid()

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.5)

While calcualting parameter "a" according to Yimsiri and Soga using the fabrictensor and the eigenvalues some major jumps in "a" can be observed.  

### Fabrictensor and Eigenvalues

In [ ]:
plt.figure(figsize = (15,7))
plt.subplot(1,2,1)
plt.plot(yimsiri_soga_a_value.cycle_number,yimsiri_soga_a_value.fabrictensor1, color = 'purple', lw = 2, ls = ':', label = 'fabrictensor[0,0]')
plt.plot(yimsiri_soga_a_value.cycle_number,yimsiri_soga_a_value.fabrictensor2, color = 'purple', lw = 2, ls = '--', label = 'fabrictensor[1,1]')
plt.plot(yimsiri_soga_a_value.cycle_number,yimsiri_soga_a_value.fabrictensor3, color = 'purple', lw = 2, label = 'fabrictensor[2,2]')
plt.legend(loc = 'center right')
plt.xlabel('Cycle number (N)', fontsize = LBF)
plt.ylabel('a value from Yimsiri & Soga (2010)', fontsize = LBF)
plt.gca().tick_params(which='major', labelsize=TS)
plt.title('"a" using fabrictensor')
plt.xlim(0,)
plt.grid()

plt.subplot(1,2,2)
plt.plot(yimsiri_soga_a_value.cycle_number,yimsiri_soga_a_value.eigenvalue1, color = 'darkcyan', lw = 2, ls = ':', label = 'eigenvalue[0]')
plt.plot(yimsiri_soga_a_value.cycle_number,yimsiri_soga_a_value.eigenvalue2, color = 'darkcyan', lw = 2, ls = '--', label = 'eigenvalue[1]')
plt.plot(yimsiri_soga_a_value.cycle_number,yimsiri_soga_a_value.eigenvalue3, color = 'darkcyan', lw = 2, label = 'eigenvalue[2]')
plt.legend(loc = 'center right')
plt.xlabel('Cycle number (N)', fontsize = LBF)
plt.ylabel('a value from Yimsiri & Soga (2010)', fontsize = LBF)
plt.gca().tick_params(which='major', labelsize=TS)
plt.title('"a" using eigenvalues')
plt.xlim(0,)
plt.grid()

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.5)

From the above figure its is clear that eigenvalues are not in oder where a in the major direction should be eigenvalue[2]

In [ ]:
CNfabrictensor

The fabric tensor for the last contact dump file demonstartes that the values that are nor located along the diagonal are small. <br>
Therefore, it should be ok to use the fabric tensor to calculate parameter "a". <br>
An alternative would be to sort the eigen values.

## Alternative
This alternative consisted in sorting the eigenvalues. <br>
However, this alternative did not work. 

In [ ]:
def get_a_yimsiri_soga_sorted(CNfabrictensor: np.array) -> float:
    """
    This function defines a as defined in Yimsiri and Soga (2010)
    DEM analysis of soil fabric effects on behaviour of sand
    """
    # Getting Eigenvalues from fabric tensor
    EigenValues = np.linalg.eig(CNfabrictensor)
    EigenValues = EigenValues[0] # Only want the eigenvalues and not the associated eigenvectors
    EigenValues = np.sort(EigenValues)
    
    # Equations using eigenvalues 
    a1 = symbols('a1')
    a2 = symbols('a2')
    a3 = symbols('a3')   
    eq1 = Eq((3*a1-5)/(5*(a1-3))-EigenValues[0])
    eq2 = Eq((3*a2-5)/(5*(a2-3))-EigenValues[1])
    eq3 = Eq((-(5+a3))/(5*(a3-3))-EigenValues[2])
    
    # Solving equations for a
    a_val1 = solve(eq1)[0]
    a_val2 = solve(eq2)[0]
    a_val3 = solve(eq3)[0]
    
    # Getting average value of a
    a_EVsorted = (a_val1 + a_val2 + a_val3)/3

    return a_EVsorted

In [ ]:
a_EVsorted = []

# Iterating trough dump files
for contact_file in yimsiri_soga_a_value.file_name:

    # Loading contact file
    contact_data = pd.read_csv(
            contact_file,
            skiprows = 9,
            delimiter = ' ',
            index_col = False,
            header = None
        )

    # Getting Fabric Tensor
    CNfabrictensor = get_fabric_tensor_cn(contact_data)

    # Getting mean a
    a_EV2 = get_a_yimsiri_soga_sorted(CNfabrictensor)
    
    # Appending to list 
    a_EVsorted.append(a_EV2)

yimsiri_soga_a_value['a_EVsorted'] = a_EVsorted

In [ ]:
plt.figure(figsize = (7,7))
plt.plot(yimsiri_soga_a_value.cycle_number,yimsiri_soga_a_value.a_EVsorted, color = 'black', lw = 2)
plt.xlabel('Cycle number (N)', fontsize = LBF)
plt.ylabel('a value from Yimsiri & Soga (2010)', fontsize = LBF)
plt.gca().tick_params(which='major', labelsize=TS)
plt.title('"a" using sorted eigenvalues')
plt.xlim(0,)
plt.grid()